<a href="https://colab.research.google.com/github/Hamoda-dabbit/Mining---classification-in-Arabic-Article/blob/main/RandomForest/RandomForest_TFidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Classification by Random Forest and TfIdf Vectorizer**

----------

 import library

  إستيراد المكتبات

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


--------

**1- Import Data set**

In [2]:
# df = pd.read_excel("drive/MyDrive/mining/1.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/2.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/3.xlsx")
df = pd.read_excel("drive/MyDrive/mining/4.xlsx")

------
shuffle  
بعثرة البيانات

In [3]:
df=shuffle(df)
df

,text,targe
23545,أخبارنا المغربية أفاد تقرير للاتحاد الأوروبي ا...,2
7439,تطالب أحزاب الخضر الأوروبية وخاصة الفرنسية بزع...,2
25339,أخبارنا المغربية سناء الوردي بعد النجاح الكبير...,2
43211,NaN,2
50219,NaN,4
...,...,...
57814,ادعى رفض محام تسلم محضر تبليغ في نزاع حول عقار...,1
68351,جرحى وإغماء عشرات الفتيات وبهدلة رجال الأمن لا...,4
67801,تضم السباحة والدراجات والجري وانتعشت بالمغرب ب...,4
28125,ستقبل ملك الأردن عبدالله الثاني اليوم السبتالط...,3


------
**2- Clean Data**

تنظيف البيانات

In [4]:
print(np.sum(df.isnull().any(axis=1)))

clean_df= df.dropna()
print(np.sum(clean_df.isnull().any(axis=1)))
#-------------
print (df.notnull().any(axis = 0))
print(len(df))
#-------------
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
      df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
        
remove_hashtag(clean_df)
clean_df
#-------------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#-------------
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
#-------------    
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#-------------
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc

stemmer = ISRIStemmer()
clean_df["text"] = clean_df['text'].apply(lambda x: processDocument(x, stemmer))
print(clean_df)

378
0
text     True
targe    True
dtype: bool
68690
                                                    text  targe
23545  اخبارنا المغربيه افاد تقرير لاتحاد الاوروبي ال...      2
7439   تطالب احزاب الخضر الاوروبيه وخاصه الفرنسيه بزع...      2
25339  اخبارنا المغربيه سناء الوردي بعد النجاح الكبير...      2
11073  ثبت حزب الاستقلال حميد شباط امينه العام وكيلا ...      3
35486  منشطه زينب صابر قالت انها سعيده بما تقدمه في ت...      0
...                                                  ...    ...
57814  ادعي رفض محام تسلم محضر تبليغ في نزاع حول عقار...      1
68351  جرحي واغماء عشرات الفتيات وبهدله رجال الامن لا...      4
67801  تضم السباحه والدراجات والجري وانتعشت بالمغرب ب...      4
28125  ستقبل ملك الاردن عبداله الثاني اليوم السبتالطي...      3
8838   تكاد احلام سكان جه سوس في اختصار المسافات الكي...      2

[68312 rows x 2 columns]


----------------------
**3- Feature Extraction:** TfIdtVectorizer

استخراج الميزات(تحويل النص إلى أرقام)

In [5]:
y = clean_df['targe']
X = clean_df['text']
stopwords_list = stopwords.words('arabic')

tfidf = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7,token_pattern=r"(?u)\b\w\w+\b",stop_words=stopwords_list)

y1 = clean_df['targe']
X1 = clean_df['text']
X1= tfidf.fit_transform (X1) .toarray ()


---------
accuracy calculation function

عمل تابع لحساب الدقة

In [6]:
def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

----------------------

**4- classification:** Random Forest

تصنيف البيانات

with Cross validation

لتوزيع عينات الاختبار والتأكد من عدم وجود

Overfitting

In [7]:
%%time
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import RandomForestClassifier

strtfdKFold = StratifiedKFold(n_splits=5)
kfold = strtfdKFold.split(X1, y1)
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
result_RandomForestClassifier=[]
#
for k, (train_index, test_index) in enumerate(kfold):
    X_train, X_test = X1[train_index], X1[test_index]
    y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]
    classifier.fit(X_train, y_train)
    y_pred=(classifier.predict(X_test))
    result_RandomForestClassifier.append(calculate_results(y_test, y_pred))


CPU times: user 1h 8min 7s, sys: 18.2 s, total: 1h 8min 26s
Wall time: 1h 8min 12s


حساب متوسط النتائج للـ5 مراحل من

 cross validation 

In [8]:
accuracy=0
f1=0
precision=0
recall=0
for i in result_RandomForestClassifier:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(result_RandomForestClassifier)
f1=f1/len(result_RandomForestClassifier)
precision=precision/len(result_RandomForestClassifier)
recall=recall/len(result_RandomForestClassifier)
# (result_DecisionTreeClassifier)
results_RandomForestClassifier=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
results_RandomForestClassifier

[{'accuracy': 92.71870192194521,
  'f1': 0.927235869172043,
  'precision': 0.927458211726609,
  'recall': 0.9271870192194521}]

--------
-----
---------
**5- Show Results**

النتائج

نتائج التدريب لكل قاعدة بيانات

In [ ]:
print('RandomForestClassifier results in dataset 1:')
results_RandomForestClassifier

RandomForestClassifier results in dataset 1:


[{'accuracy': 94.79871052663006,
  'f1': 0.948054790221879,
  'precision': 0.9484085086796259,
  'recall': 0.9479871052663006}]

In [ ]:
print('RandomForestClassifier results in dataset 2:')
results_RandomForestClassifier

RandomForestClassifier results in dataset 2:


[{'accuracy': 94.35977276279998,
  'f1': 0.9436010724434535,
  'precision': 0.9437771120290449,
  'recall': 0.9435977276279998}]

In [ ]:
print('RandomForestClassifier results in dataset 3:')
results_RandomForestClassifier

RandomForestClassifier results in dataset 3:


[{'accuracy': 93.20200229777969,
  'f1': 0.9320751779523017,
  'precision': 0.9322813561648182,
  'recall': 0.9320200229777968}]

In [ ]:
print('RandomForestClassifier results in dataset 4:')
results_RandomForestClassifier

RandomForestClassifier results in dataset 4:


[{'accuracy': 92.80506484204825,
  'f1': 0.9280963000587084,
  'precision': 0.9283160067681553,
  'recall': 0.9280506484204825}]

In [9]:
Final_Result=[
  {'accuracy': 94.79871052663006,
  'f1': 0.948054790221879,
  'precision': 0.9484085086796259,
  'recall': 0.9479871052663006}         
  ,
  {'accuracy': 94.35977276279998,
  'f1': 0.9436010724434535,
  'precision': 0.9437771120290449,
  'recall': 0.9435977276279998}
  ,
  {'accuracy': 93.20200229777969,
  'f1': 0.9320751779523017,
  'precision': 0.9322813561648182,
  'recall': 0.9320200229777968}
  ,
  {'accuracy': 92.80506484204825,
  'f1': 0.9280963000587084,
  'precision': 0.9283160067681553,
  'recall': 0.9280506484204825}  
  ]

accuracy=0
f1=0
precision=0
recall=0
for i in Final_Result:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(Final_Result)
f1=f1/len(Final_Result)
precision=precision/len(Final_Result)
recall=recall/len(Final_Result)
# (result_DecisionTreeClassifier)
Final_Result=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
print("RAM: 3.12 , CPU times: user 1h 8min 7s")
print('Final_Result:')
print(Final_Result)

RAM: 3.12 , CPU times: user 1h 8min 7s
Final_Result:
[{'accuracy': 93.79138760731449, 'f1': 0.9379568351690857, 'precision': 0.9381957459104111, 'recall': 0.9379138760731449}]
